## Project 10, task 3, Кривоногов Н.В. (hemerahu)

## 1. Создай соединение с базой данных с помощью библиотеки sqlite3.

In [1]:
import sqlite3

conn = sqlite3.connect('../data/checking-logs.sqlite')

conn

## 2. Создай новую таблицу datamart в базе данных, объединив таблицы pageviews и checker с помощью только одного запроса.
- Таблица должна содержать следующие столбцы: uid, labname, first_commit_ts, first_view_ts.
- first_commit_ts — это просто новое имя для столбца timestamp из таблицы checker; он показывает первый коммит конкретного лабораторного задания конкретного пользователя.
- first_view_ts — первое посещение пользователем из таблицы pageviews, метка времени посещения пользователем ленты новостей.
- По-прежнему нужно использовать фильтр status = 'ready'.
- По-прежнему нужно использовать фильтр numTrials = 1.
- Имена лабораторных заданий по-прежнему должны быть из следующего списка: laba04, laba04s, laba05, laba06, laba06s, project1.
- Таблица должна содержать только пользователей (uid с user_*), а не администраторов.
- first_commit_ts и first_view_ts должны быть распарсены как datetime64[ns].

In [2]:
import pandas as pd

datamart = pd. io.sql.read_sql('SELECT c.uid, c.labname, c.timestamp AS first_commit_ts, MIN(p.datetime) AS first_view_ts \
                                FROM checker AS c LEFT JOIN pageviews AS p ON c.uid = p.uid \
                                WHERE c.status = "ready" AND c.numTrials = 1 \
                                AND c.labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s", "project1") \
                                AND c.uid LIKE "user_%" GROUP BY c.timestamp, c.uid;', 
                                conn, parse_dates=['first_commit_ts', 'first_view_ts'])

datamart.to_sql('datamart', conn, if_exists='replace')

140

## 3. Используя методы библиотеки Pandas, создай два датафрейма: test и control.
- test должен включать пользователей, у которых имеются значения в first_view_ts.
- control должен включать пользователей, у которых отсутствуют значения в first_view_ts.
- Замени пропущенные значения в control средним значением first_view_ts пользователей из test (оно пригодится нам для анализа в будущем).
- Сохрани обе таблицы в базе данных (вы будете использовать их в следующих упражнениях).

In [3]:
test = datamart.dropna().copy()

control = datamart[datamart['first_view_ts'].isna()].copy()
control['first_view_ts'] = test['first_view_ts'].mean()

test.to_sql('test', conn, if_exists='replace')
control.to_sql('control', conn, if_exists='replace')

81

## 4. Закрой соединение.

In [4]:
conn.close()